In [1]:
import pandas as pd 
import numpy as np 
import os 
import json 

#raw_path = "/home/ihsan/Documents/thesis_generator/results/devin/to_process/" #needs the absolute path, no tildes! 
#processed_path = "/home/ihsan/Documents/thesis_generator/results/devin"

#usb drive
#raw_path = '/media/ihsan/LID_FLASH_1/Thesis/thesis_generator/results/run_2/'

raw_path = "/home/ihsan/Documents/thesis_generator/results/to_process/"

processed_path = "/home/ihsan/Documents/thesis_models/unsplit"
#processed_path = '/media/ihsan/LID_FLASH_1/Thesis/thesis_generator/results/run_2/processed/'
items = os.listdir(raw_path)
items.sort()
print(type(items))
for file in items:
    if ('.csv') not in str(file):
        del items[items.index(file)]   
print(items)

seq_length_dict = {}
seq_length_dict_filename = processed_path + "/sequence_lengths.json"
#suffix = "3a"
#csv_path = "~/Documents/thesis_generator/results/devin/crack_growth_sequence" + suffix + ".csv"
sequence_lengths = {} #save sequence lengths as a dict. or maybe a json?

<type 'list'>
['crack_growth_sequence1c_0.csv', 'crack_growth_sequence1c_1.csv', 'crack_growth_sequence1c_2.csv', 'crack_growth_sequence1c_3.csv', 'crack_growth_sequence1c_4.csv', 'crack_growth_sequence1c_5.csv']


In [2]:
j = 0 #counter. 
for file in items:
    print(str(file))
    csv_path = raw_path + str(file)
    if ("_0.") in str(file): #only the first file in the series has a header. 
        cg_seq_df = pd.read_csv(csv_path)
        header_names = cg_seq_df.columns.values
    else:
        cg_seq_df = pd.read_csv(csv_path,names=header_names)
        print(cg_seq_df.columns.values)

    train_list = ['StepIndex','delta_K_current_1','ctip_posn_curr_1','delta_K_current_2','ctip_posn_curr_2',
          'delta_K_current_3','ctip_posn_curr_3','delta_K_current_4','ctip_posn_curr_4','Load_1','Load_2'] #and seq_id,somehow 

    label_list = ['StepIndex','delta_a_current_1','delta_a_current_2','delta_a_current_3','delta_a_current_4']

    train_df = cg_seq_df[train_list]
    label_train_df = cg_seq_df[label_list]

    #labels_csv_path = "~/Documents/thesis_generator/results/devin/label" + suffix + "_headers"+ ".csv"
    #train_csv_path = "~/Documents/thesis_generator/results/devin/train" + suffix + "_headers" + ".csv"

    indices = train_df[train_df['StepIndex']==1].index.tolist()
    indices.append(train_df.shape[0]-1)
    indices_offset_min1 = [i-1 for i in indices]
    print("file {}'s indices_offset_min1 {}".format(str(file),indices_offset_min1))
    indices_offset_min1.pop(0)
    print("indices: {}, indices_offset_min1: {}".format(indices,indices_offset_min1))

    ranges = [(t,s) for t,s in zip(indices,indices_offset_min1)]
    #print("before changing :{}".format(ranges))
    '''for tuple in ranges:
        print(tuple)
        tuple[1:][0] = (tuple[1:][0]) + 1'''

    #ranges[1:][0] = ranges[1:][0] + 1
    print("\nafter changing :{} ".format(ranges))
    #print("lengths: {} ".format([indices[4]-indices[3],indices[3]-indices[2],indices[2]-indices[1],indices[1]-indices[0]]))
    print("lengths: {} ".format([t-s for (s,t) in ranges]))
    
    i = 0
    for indices_as_tuples in ranges:
        i=i+1
        print("indices as tuples: {}".format(indices_as_tuples))
        train_df_as_np_array = train_df[indices_as_tuples[0]:indices_as_tuples[1]].values
        label_train_df_as_np_array = label_train_df[indices_as_tuples[0]:indices_as_tuples[1]].values
        print("df_as_np_array shape: {}".format(train_df_as_np_array.shape))
        print("file: {}".format(file))
        identifier = str(str(file)[-8:-6]) #eg 1a 2a etc. #you can use a regex. 
        print("identifier: {}".format(identifier))
        
        #j is sequence id. #i is the sequence number within the csv. 
        np_train_path = processed_path + "/sequence_" + identifier + "_" + str(j) + "_" + str(i) + ".npy"
        np_label_train_path = processed_path + "/sequence_" + identifier + "_" + str(j) + "_" + str(i) + "_label_.npy"
        seq_length_dict["sequence_" + identifier + "_" + str(j) + "_" + str(i)] = indices_as_tuples[1] - indices_as_tuples[0]
        #seq_length_dict = json.load(open(seq_length_dict))
        #UNCOMMENT THESE IF YOU WANT TO SAVETHE ARRAYS!!!
        print("np_train_path: {}".format(np_train_path))
        print("np_label_train_path :{}".format(np_label_train_path))
        np.save(np_train_path,train_df_as_np_array)
        np.save(np_label_train_path,label_train_df_as_np_array)
        j= j + 1
        
print(seq_length_dict)
json.dump(seq_length_dict, open(seq_length_dict_filename, 'wb'))

crack_growth_sequence1c_0.csv
file crack_growth_sequence1c_0.csv's indices_offset_min1 [-1, 83554, 167163, 250704, 334236, 417787, 501302, 584803, 668284, 751817, 835270]
indices: [0, 83555, 167164, 250705, 334237, 417788, 501303, 584804, 668285, 751818, 835271], indices_offset_min1: [83554, 167163, 250704, 334236, 417787, 501302, 584803, 668284, 751817, 835270]

after changing :[(0, 83554), (83555, 167163), (167164, 250704), (250705, 334236), (334237, 417787), (417788, 501302), (501303, 584803), (584804, 668284), (668285, 751817), (751818, 835270)] 
lengths: [83554, 83608, 83540, 83531, 83550, 83514, 83500, 83480, 83532, 83452] 
indices as tuples: (0, 83554)
df_as_np_array shape: (83554, 11)
file: crack_growth_sequence1c_0.csv
identifier: 1c
np_train_path: /home/ihsan/Documents/thesis_models/unsplit/sequence_1c_0_1.npy
np_label_train_path :/home/ihsan/Documents/thesis_models/unsplit/sequence_1c_0_1_label_.npy
indices as tuples: (83555, 167163)
df_as_np_array shape: (83608, 11)
file: c

['Unnamed: 0' 'StepIndex' 'direction_1' 'delta_a_current_1'
 'delta_K_prev_1' 'delta_K_current_1' 'ctip_posn_curr_1' 'ctip_posn_prev_1'
 'crack_length_last_overload_1' 'crack_length_1' 'K_last_overload_1'
 'pz_rad_curr_1' 'pz_rad_prev_1' 'crack_id_1' 'hole_1' 'direction_2'
 'delta_a_current_2' 'delta_K_prev_2' 'delta_K_current_2'
 'ctip_posn_curr_2' 'ctip_posn_prev_2' 'crack_length_last_overload_2'
 'crack_length_2' 'K_last_overload_2' 'pz_rad_curr_2' 'pz_rad_prev_2'
 'crack_id_2' 'hole_2' 'direction_3' 'delta_a_current_3' 'delta_K_prev_3'
 'delta_K_current_3' 'ctip_posn_curr_3' 'ctip_posn_prev_3'
 'crack_length_last_overload_3' 'crack_length_3' 'K_last_overload_3'
 'pz_rad_curr_3' 'pz_rad_prev_3' 'crack_id_3' 'hole_3' 'direction_4'
 'delta_a_current_4' 'delta_K_prev_4' 'delta_K_current_4'
 'ctip_posn_curr_4' 'ctip_posn_prev_4' 'crack_length_last_overload_4'
 'crack_length_4' 'K_last_overload_4' 'pz_rad_curr_4' 'pz_rad_prev_4'
 'crack_id_4' 'hole_4' 'SeqID' 'NoiseSig' 'Load_1' 'Load_2'

['Unnamed: 0' 'StepIndex' 'direction_1' 'delta_a_current_1'
 'delta_K_prev_1' 'delta_K_current_1' 'ctip_posn_curr_1' 'ctip_posn_prev_1'
 'crack_length_last_overload_1' 'crack_length_1' 'K_last_overload_1'
 'pz_rad_curr_1' 'pz_rad_prev_1' 'crack_id_1' 'hole_1' 'direction_2'
 'delta_a_current_2' 'delta_K_prev_2' 'delta_K_current_2'
 'ctip_posn_curr_2' 'ctip_posn_prev_2' 'crack_length_last_overload_2'
 'crack_length_2' 'K_last_overload_2' 'pz_rad_curr_2' 'pz_rad_prev_2'
 'crack_id_2' 'hole_2' 'direction_3' 'delta_a_current_3' 'delta_K_prev_3'
 'delta_K_current_3' 'ctip_posn_curr_3' 'ctip_posn_prev_3'
 'crack_length_last_overload_3' 'crack_length_3' 'K_last_overload_3'
 'pz_rad_curr_3' 'pz_rad_prev_3' 'crack_id_3' 'hole_3' 'direction_4'
 'delta_a_current_4' 'delta_K_prev_4' 'delta_K_current_4'
 'ctip_posn_curr_4' 'ctip_posn_prev_4' 'crack_length_last_overload_4'
 'crack_length_4' 'K_last_overload_4' 'pz_rad_curr_4' 'pz_rad_prev_4'
 'crack_id_4' 'hole_4' 'SeqID' 'NoiseSig' 'Load_1' 'Load_2'